In [ ]:
# load libraries and scripts
import JSON
using Plots
using HDF5
using Trapz
include("../scripts/tools.jl")
include("../scripts/get_grid.jl")
#include("../scripts/get_params.jl")
include("../scripts/disturbances.jl")

In [ ]:
#load borders
border = import_border("../data/border.json")

# create the lattice grid
dx = 20
Nx, Ny, xrange, yrange, isinside, isborder, n = get_grid(border, Float64(dx))

# define x and y coordinate for plotting
x = repeat(reshape(xrange,1,Nx), Ny, 1)
y = repeat(reshape(yrange,Ny,1), 1, Nx);

In [ ]:
    data = h5read("../data/pantagruel.h5", "/")
    mg = vec(data["gen_inertia"])
    dg = vec(data["gen_prim_ctrl"])
    idgen = Int64.(vec(data["gen"][:, 1]))
    coord = alberts_projection(
        data["bus_coord"] ./ (180 / pi),
        13.37616 / 180 * pi,
        46.94653 / 180 * pi,
        10 / 180 * pi,
        50 / 180 * pi,
    )
    
    isgrid = isinside .| isborder
    dl = vec(data["load_freq_coef"])
    idb = Int64.(data["branch"][:, 1:2])
    bline = 1 ./ data["branch"][:, 4]

    dem = vec(data["bus"][:, 3]) / 100
    gen = vec(data["gen"][:, 2]) / 100
    
    Nx = length(xrange)
    Ny = length(yrange)
    
    idin = findall(isinside)
    Nbus = length(idin)
    def_val = 1E-2
    m = def_val * ones(Nbus)
    d = def_val * ones(Nbus)
    pl = def_val * ones(Nbus)
    pg = def_val * ones(Nbus)
    lat_coord = zeros(Nbus, 2)
    for i = 1:Nbus
        lat_coord[i, :] = [yrange[idin[i][1]], xrange[idin[i][2]]]
    end
    println(maximum(mg))
    for g in 1:length(idgen)
        k = argmin((lat_coord[:, 2] .- coord[idgen[g],1]).^2 +
            (lat_coord[:, 1] .- coord[idgen[g],2]).^2)
        #println(k)
        m[k] += mg[g]
        d[k] += dg[g]
        pg[k] += gen[g]
    end
    
    for l in 1:length(dl)
        k = argmin((lat_coord[:, 2] .- coord[l,1]).^2 +
            (lat_coord[:, 1] .- coord[l,2]).^2)
        d[k] += dl[l]
        pl[k] += dem[l]
    end

    mlat = zeros(Ny, Nx)
    dlat = zeros(Ny, Nx)
    pglat = zeros(Ny, Nx)
    pllat = zeros(Ny, Nx)
    mlat[isinside] = m
    dlat[isinside] = d
    pglat[isinside] = pg
    pllat[isinside] = pl

kappa = 10
tau =  # for heat equation tau=kappa*dt/dx^2
interval = 100
dt = 0.01
Niter = 50000
mlat_new = zeros(size(mlat))
dlat_new = zeros(size(dlat))
pglat_new = zeros(size(mlat))
pllat_new = zeros(size(dlat))
@time begin
    for k in 1:Niter
        
        if(mod(k,interval) == 0)
            temp = copy(mlat)
        end
        
        Threads.@threads for i in 2:Ny-1
            Threads.@threads for j in 2:Nx-1
                if(isinside[i,j])
                    mlat_new[i,j] = (1.0 - 4.0 * tau) * mlat[i,j] + tau * (mlat[i+1,j] +
                        mlat[i-1,j] + mlat[i,j+1] + mlat[i,j-1])
                    dlat_new[i,j] = (1.0 - 4.0 * tau) * dlat[i,j] + tau * (dlat[i+1,j] +
                        dlat[i-1,j] + dlat[i,j+1] + dlat[i,j-1])
                    pglat_new[i,j] = (1.0 - 4.0 * tau) * pglat[i,j] + tau * (pglat[i+1,j] +
                        pglat[i-1,j] + pglat[i,j+1] + pglat[i,j-1])
                    pllat_new[i,j] = (1.0 - 4.0 * tau) * pllat[i,j] + tau * (pllat[i+1,j] +
                        pllat[i-1,j] + pllat[i,j+1] + pllat[i,j-1])
                end
            end
        end
        Threads.@threads for k in 1:size(n,1)
            i = Int64(n[k,1])
            j = Int64(n[k,2])
            nx = n[k,4]
            ny = n[k,3]
            if(nx == 1)
                mlat_new[i,j] = mlat_new[i,j-2]
                dlat_new[i,j] = dlat_new[i,j-2]
                pglat_new[i,j] = pglat_new[i,j-2]
                pllat_new[i,j] = pllat_new[i,j-2]
            elseif(nx == -1)
                mlat_new[i,j] = mlat_new[i,j+2]
                dlat_new[i,j] = dlat_new[i,j+2]
                pglat_new[i,j] = pglat_new[i,j+2]
                pllat_new[i,j] = pllat_new[i,j+2]
            end
            if(ny == 1)
                mlat_new[i,j] = mlat_new[i-2,j]
                dlat_new[i,j] = dlat_new[i-2,j]
                pglat_new[i,j] = pglat_new[i-2,j]
                pllat_new[i,j] = pllat_new[i-2,j]
            elseif(ny == -1)
                mlat_new[i,j] = mlat_new[i+2,j]
                dlat_new[i,j] = dlat_new[i+2,j]
                pglat_new[i,j] = pglat_new[i+2,j]
                pllat_new[i,j] = pllat_new[i+2,j]
            end
        end
        mlat = copy(mlat_new)
        dlat = copy(dlat_new)
        pglat = copy(pglat_new)
        pllat = copy(pllat_new)
        
        if(mod(k,interval) == 0)
            #println([k maximum(abs.(mlat-temp))])
            println([k sum(mlat[isinside])])
        end
    end
end


In [ ]:
println(maximum(mlat[isinside]))
println(minimum(mlat[isinside]))
contour(pllat, fill=true)
#contour(1 ./ mlat, fill=false)

In [ ]:
contour(pglat, fill=true)

In [ ]:
contour(mlat, fill=true)

In [ ]:
contour(dlat, fill=true)

In [ ]:
contour(dlat ./ mlat, fill=true)

In [ ]:
contour(pglat - pllat, fill=true)